In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime
import time
import random

def search_book_reviews(book_name, author):
    # Combine book name and author for search query
    search_query = f"{book_name} {author} book review"
    
    # Perform Google search
    url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract search results
    search_results = soup.find_all('div', class_='g')
    
    reviews = []
    
    for result in search_results:
        link = result.find('a')
        if link:
            review_url = link['href']
            
            # Skip non-http links and known non-review sites
            if not review_url.startswith('http') or any(site in review_url for site in ['google.com', 'wikipedia.org']):
                continue
            
            try:
                # Fetch the review page
                review_response = requests.get(review_url, headers=headers, timeout=10)
                review_soup = BeautifulSoup(review_response.text, 'html.parser')
                
                # Extract review text (this is a simplistic approach and may need refinement)
                review_text = ' '.join([p.text for p in review_soup.find_all('p')])
                
                # Extract date (this is a placeholder, as date formats vary widely)
                date = review_soup.find('time')
                review_date = date['datetime'] if date else 'Unknown'
                
                reviews.append({
                    'review_text': review_text[:500],  # Limit to first 500 characters
                    'review_date': review_date,
                    'review_website': review_url
                })
                
            except Exception as e:
                print(f"Error processing {review_url}: {str(e)}")
            
            # Be polite to servers
            time.sleep(random.uniform(1, 3))
    
    # Create DataFrame
    df = pd.DataFrame(reviews)
    
    return df

# Example usage
book_name = "To Kill a Mockingbird"
author = "Harper Lee"

df = search_book_reviews(book_name, author)

# Save to CSV
df.to_csv(f"{book_name.replace(' ', '_')}_reviews.csv", index=False)
print(f"Reviews saved to {book_name.replace(' ', '_')}_reviews.csv")